# The DyNet computation graph
Computation graphs are the fundamental language used in frameworks like TensorFlow, Theano, PyTorch, ... They describe a graph of mathematical operations. A computation graph is defined by a set of nodes and edges, where edges represent data (for example, a scalar value representing an input to the model or a matrix representing a set of learnable parameters) and nodes represent function calls (computations) (for example, multiplying a value by another value).

The [class notes](http://www.cs.cornell.edu/courses/cs5740/2018sp/lectures/04-nn-compgraph.pdf) contain more information about computation graphs.

## Entering/exiting the computation graph
Below we learn how to put input data into the computation graph and perform a forward pass to get output data back.

In [ ]:
import dynet as dy
import numpy as np

my_scalar = np.random.randint(0,100)
my_vector = np.random.random([3])
my_matrix = np.random.random([3,3])

In [ ]:
my_scalar

In [ ]:
my_vector

In [ ]:
my_matrix

Now that we have some random data, let's put it into the DyNet computation graph. First, we have to renew the computation graph.

A computational graph consists entirely of expressions, and it can't accept Numpy ndarrays or Python scalars. To input our data into a computational graph, we use the functions `scalarInput`, `inputVector` and `inputTensor` to create input expressions for each of the type of inputs.

In [ ]:
dy.renew_cg()

scalar_exp = dy.scalarInput(my_scalar)
vector_exp = dy.inputVector(my_vector)
matrix_exp = dy.inputTensor(my_matrix)

Let's look at the size of these expressions using the [`dim`](http://dynet.readthedocs.io/en/latest/python_ref.html#dynet.Expression.dim) function.

In [ ]:
scalar_exp.dim()

In [ ]:
vector_exp.dim()

In [ ]:
matrix_exp.dim()

Indeed these inputs are a type of Expression, and can be used in a DyNet computation graph.

In [ ]:
print(type(matrix_exp))
isinstance(matrix_exp, dy.Expression)

Let's get the data back by calling the [`value`](http://dynet.readthedocs.io/en/latest/python_ref.html#dynet.Expression.value) function. Depending on the dimensions of your expression, it might return a float (if it's a scalar), a list (if it's a vector), or a numpy array (if it's a matrix).

In [ ]:
scalar_exp.value()

In [ ]:
vector_exp.value()

In [ ]:
matrix_exp.value()

You can also create DyNet input expressions of any size by using several functions provided by DyNet, including [`zeros`](http://dynet.readthedocs.io/en/latest/python_ref.html#dynet.zeros), [`ones`](http://dynet.readthedocs.io/en/latest/python_ref.html#dynet.ones), and sampling from various random distributions.

In [ ]:
# Creating an expression containing matrix of zeros with DyNet
zeros_exp1 = dy.zeros((3, 3))
zeros_exp1.value()

# Doing the same with Numpy
zeros_np = np.zeros((3, 3))
zeros_exp2 = dy.inputTensor(zeros_np)

# The result is the same
assert((zeros_exp1.value() == zeros_exp2.value()).all())

In [ ]:
random_uniform = dy.random_uniform((3, 3), -1.0, 1.0)
random_uniform.value()

## Basic mathematic operators

Now we will learn about some of the basic math operators DyNet provides. We will also look at forward passes in the graph.

It supports basic operators like exponentiating, trigonometric functions, and nonlinearities on any expressions.

In [ ]:
matrix_exp.value()

In [ ]:
dy.exp(matrix_exp).value()

In [ ]:
dy.tanh(matrix_exp).value()

In [ ]:
# ReLU activation
dy.rectify(matrix_exp).value()

Any time `value` or `forward` is called, a forward pass is performed on the graph. This means that the  computations are actually carried out and a numerical value is returned. All nodes and edges in the graph that contribute to the value you request will be used. Before calling either of these functions, the graph is just a set of nodes and edges describing a computation. We will discuss forward passes more in detail during the batching section.

We can check that without the `value` call, the result is not numeric, it is an `Expression`:

In [ ]:
expr = dy.rectify(matrix_exp)
a = expr * 10
a.value()

In [ ]:
expr.value()

DyNet has some simple binary operators overloaded, including +, -, \*, and /. This means you can perform any of these operations with an Expression and a Python scalar, and the operation will be projected across all dimensions of the expression.

In [ ]:
(matrix_exp + 1.0).value()

In [ ]:
(matrix_exp / 2.0).value()

It also provides component-wise operations on multiple expressions:

In [ ]:
(matrix_exp + random_uniform).value()

In [ ]:
dy.cdiv(matrix_exp, random_uniform).value()

When performing element-wise operations, you need to make sure the shapes match. DyNet automatically performs broadcasting.

In [ ]:
(dy.ones((3,4)) + dy.ones((3,1))).value()

Some operations are useful for summarizing information about an Expression or reshaping it.

In [ ]:
dy.sum_elems(matrix_exp).value()

In [ ]:
dy.mean_elems(vector_exp).value()

In [ ]:
dy.reshape(matrix_exp, (9, 1)).dim()

A few special operations can be used on lists of expressions. `esum` performs an element-wise sum on a list of expressions. This is useful for summing loss values for multiple training examples.

In [ ]:
dy.esum([matrix_exp, random_uniform]).value()

## Parameter Collections and Parameters
DyNet has a [`ParameterCollection`](http://dynet.readthedocs.io/en/latest/python_ref.html#parametercollection-and-parameters) object which is used to store optimizable tensors (e.g., a bias vector or weight matrix). 

[`Parameters`](http://dynet.readthedocs.io/en/latest/python_ref.html#dynet.ParameterCollection) is a subclass of `Expresion` that contains optimizable tensor data. (http://dynet.readthedocs.io/en/latest/python_ref.html#dynet.parameter).

[`LookupParameters`](http://dynet.readthedocs.io/en/latest/python_ref.html#dynet.LookupParameters) represents a table of parameters. In general, these are used as lists of vectors, where you can look up the appropriate vector and add it to the graph by indexing the lookup parameters as you would a normal Python list.

Below we will see examples of the two types of parameters. First, we have to create the parameter collection.

In [ ]:
pc = dy.ParameterCollection()

Then we can create a parameters object. Let's make a weight vector object that we can multiply out matrices with. When calling `add_parameters`, the parameters are automatically added to the computation graph, as well as stored in the `ParameterCollection`.

In [ ]:
weights = pc.add_parameters((3, 2))
biases = pc.add_parameters((1, 2,), init=dy.UniformInitializer(0.1))

In [ ]:
weights.value()

In [ ]:
biases.value()

Above, we created a parameters vector of size 1 x 2, loaded it into the graph, and got its value. We can also check the value by calling [`as_array`](http://dynet.readthedocs.io/en/latest/python_ref.html#dynet.Parameters.as_array):

In [ ]:
biases.as_array()

We can perform a few computations in the graph:

In [ ]:
m1 = matrix_exp * weights + biases
m2 = random_uniform * weights + biases
result = dy.logistic(m1) + dy.logistic(m2)
result.value()

Recall that until we call `value`, no computations have actually been performed. 

Calling `backward` on a scalar `Expression` will perform the backward pass and compute gradients of the expression with respect to all parameters. 

In [ ]:
scalar_result = dy.sum_elems(result)
scalar_result.backward()

Now let's create some lookup parameters.

In [ ]:
lookup_parameters = pc.add_lookup_parameters((100, 3))

We get values from the lookup parameters by using syntax similar to Python indexing:

In [ ]:
lookup_vector = lookup_parameters[13]
lookup_vector.value()

Sometimes during model development it's necessary to save and load learned parameters. DyNet will save all `Parameters` and `LookupParameters` objects. However, it won't save other things during training such as learning rate coefficients and optimizer parameters, so be careful.

In [ ]:
save_filename = "save.dy"

pc.save(save_filename)
pc.populate(save_filename)